In [6]:
import sqlalchemy
from sqlalchemy import create_engine, text
from tabulate import tabulate

In [2]:
DB_USER = "myuser"
DB_PASS = "mypassword"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "mydb"

In [3]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

In [4]:
engine = create_engine(DATABASE_URL)

### Function to execute SQL statements safely

In [7]:
def execute_sql(sql_statement):
    try:
        with engine.connect() as connection:
            connection.execute(text(sql_statement))
            connection.commit()
            print(f"SQL statement executed successfully:\n{sql_statement}")
    except sqlalchemy.exc.ProgrammingError as e:
        print(f"Error executing SQL statement:\n{sql_statement}\nError: {e}")
    except Exception as e:
        print(f"An unexpected error occurred:\n{sql_statement}\nError: {e}")

### 1. Create Tables

In [8]:
execute_sql("""
    CREATE TABLE IF NOT EXISTS Employees (
        employee_id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        department VARCHAR(255)
    );
""")

SQL statement executed successfully:

    CREATE TABLE IF NOT EXISTS Employees (
        employee_id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        department VARCHAR(255)
    );



In [9]:
execute_sql("""
    CREATE TABLE IF NOT EXISTS Salaries (
        salary_id SERIAL PRIMARY KEY,
        employee_id INT REFERENCES Employees(employee_id),
        salary DECIMAL(10, 2),
        pay_date DATE
    );
""")

SQL statement executed successfully:

    CREATE TABLE IF NOT EXISTS Salaries (
        salary_id SERIAL PRIMARY KEY,
        employee_id INT REFERENCES Employees(employee_id),
        salary DECIMAL(10, 2),
        pay_date DATE
    );



### 2. Create Roles

In [14]:
execute_sql("CREATE ROLE analyst;")

SQL statement executed successfully:
CREATE ROLE analyst;


In [15]:
execute_sql("CREATE ROLE data_admin;")

SQL statement executed successfully:
CREATE ROLE data_admin;


### 3. Create Users and Grant Membership to Roles


In [17]:
execute_sql("CREATE USER analysis_user WITH PASSWORD 'secure_password';")
execute_sql("GRANT analyst TO analysis_user;")

SQL statement executed successfully:
CREATE USER analysis_user WITH PASSWORD 'secure_password';
SQL statement executed successfully:
GRANT analyst TO analysis_user;


In [18]:
execute_sql("CREATE USER admin_user WITH PASSWORD 'secure_password';")
execute_sql("GRANT data_admin TO admin_user;")

SQL statement executed successfully:
CREATE USER admin_user WITH PASSWORD 'secure_password';
SQL statement executed successfully:
GRANT data_admin TO admin_user;


### 4. Grant Privileges (Data Access)

In [19]:
# Analyst Role: Read-only access to Employees and Salaries tables
execute_sql("GRANT SELECT ON Employees, Salaries TO analyst;")

SQL statement executed successfully:
GRANT SELECT ON Employees, Salaries TO analyst;


In [20]:
# Data Admin Role: Full access to Employees table
execute_sql("GRANT SELECT, INSERT, UPDATE, DELETE ON Employees TO data_admin;")

SQL statement executed successfully:
GRANT SELECT, INSERT, UPDATE, DELETE ON Employees TO data_admin;


In [21]:
# Data Admin Role: Limited access to Salaries table (only SELECT and INSERT)
execute_sql("GRANT SELECT, INSERT ON Salaries TO data_admin;") # Removed Update, Delete

SQL statement executed successfully:
GRANT SELECT, INSERT ON Salaries TO data_admin;


### 5.  Revoke Privileges

In [23]:
# Let's say we want to prevent the analyst from seeing the employee names.
execute_sql("REVOKE SELECT (name) ON Employees FROM analyst;")

SQL statement executed successfully:
REVOKE SELECT (name) ON Employees FROM analyst;


In [24]:
# Additional Revoke Example: Remove all access to Salaries from analyst
execute_sql("REVOKE ALL PRIVILEGES ON Salaries FROM analyst;")

SQL statement executed successfully:
REVOKE ALL PRIVILEGES ON Salaries FROM analyst;


### Drop the user and role

In [25]:
execute_sql("REVOKE analyst FROM analysis_user;")
execute_sql("DROP USER IF EXISTS analysis_user;")

SQL statement executed successfully:
REVOKE analyst FROM analysis_user;
SQL statement executed successfully:
DROP USER IF EXISTS analysis_user;


In [ ]:
execute_sql("REVOKE data_admin FROM admin_user;")
execute_sql("DROP USER admin_user;")
execute_sql("DROP ROLE analyst;")
execute_sql("DROP ROLE data_admin;")

### Drop tables

In [28]:
execute_sql("DROP TABLE IF EXISTS Salaries;")
execute_sql("DROP TABLE IF EXISTS Employees;")

SQL statement executed successfully:
DROP TABLE IF EXISTS Salaries;
SQL statement executed successfully:
DROP TABLE IF EXISTS Employees;
